<a href="https://colab.research.google.com/github/karanbhatt1/ConversationalChatBot/blob/main/ArtificalIntelligenceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBECTIVE:
> To build a conversational AI which takes the input data like pdf, txt,image,json and answer the questions related to it.


# Installing the useful libraries

In [1]:
!pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.0 MB/s eta 0:00:0

In [2]:
!pip install chromadb

In [3]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 3.6 MB/s eta 0:00:00


# Importing the dependencies

In [4]:
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from pypdf import PdfReader
import re

# Step 2 -- Document Processing

## 1. Taking a plain text file from user
> Merging the text in the document.

1-> **!pip install pypdf [to handle pdf files]**
</br>
2-> **from pypdf import PdfReader**
</br>
3-> **readerObject.pages** to get toatal pages in pdf
<br>
4-> **.extract_text()**: to extract text from particular page

In [5]:
fldr = "/content/drive/MyDrive/machinelearningprojects/AIProject/NIPS-2017-attention-is-all-you-need-Paper.pdf"

In [6]:
pdfFile = PdfReader(fldr)

In [7]:
def clean_text(text):
    # Replace multiple spaces/tabs/newlines with a single space
    text = re.sub(r'[\t\n\r]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [8]:
len(pdfFile.pages)

11

In [9]:
# concatinating the text:
combinedText = ""
for page in pdfFile.pages:
  txt = page.extract_text()
  if txt:
    clntext = clean_text(txt)
    combinedText += clntext+"/n"

In [10]:
combinedText

'Attention Is All You Need Ashish Vaswani⇤ Google Brain avaswani@google.com Noam Shazeer⇤ Google Brain noam@google.com Niki Parmar⇤ Google Research nikip@google.com Jakob Uszkoreit⇤ Google Research usz@google.com Llion Jones⇤ Google Research llion@google.com Aidan N. Gomez⇤ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser⇤ Google Brain lukaszkaiser@google.com Illia Polosukhin⇤ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our mode

In [11]:
len(combinedText) # 461134

32635

# Step 3 Splitting the text
> The data is splited to chunks and converted to document using:
> .create_document([cleaned_text])

> Total chunks = 119 with size of each chunk as 5000

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [13]:
len(combinedText) # len = four lakhs sixty one thousand one hundred thirty four token or text

32635

In [14]:
def chunk_size(combinedText):
  if(len(combinedText)>250000):
    return 70000
  elif(len(combinedText)>100000):
    return 10000
  elif(len(combinedText)>50000):
    return 5000
  elif(len(combinedText)>25000):
    return 1000
  else:
    return 500

In [15]:
splitter = CharacterTextSplitter(separator='/n',
                                 chunk_size= 5000,
                                 chunk_overlap = 0,
                                 length_function = len
                                 )

In [16]:
splited_chunks = splitter.split_text(combinedText) # This can be used for small documents. Seprating based on seprator.

In [17]:
splited_chunks

['Attention Is All You Need Ashish Vaswani⇤ Google Brain avaswani@google.com Noam Shazeer⇤ Google Brain noam@google.com Niki Parmar⇤ Google Research nikip@google.com Jakob Uszkoreit⇤ Google Research usz@google.com Llion Jones⇤ Google Research llion@google.com Aidan N. Gomez⇤ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser⇤ Google Brain lukaszkaiser@google.com Illia Polosukhin⇤ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our mod

In [18]:
len(splited_chunks)

10

In [19]:
stxt = splitter.create_documents([combinedText])

In [20]:
len(stxt) # 119 chunks

10

## Using RecursiveCharacterTextSplitter
> it is used to make chunks based on size not dependent on seprator.


In [21]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,    # max 1000 characters per chunk
    chunk_overlap=10   # 200 characters overlap
)
chunks = splitter.split_text(combinedText)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 136


In [22]:
finalDocs = splitter.create_documents([combinedText])

In [23]:
len(finalDocs)

136

# Step 3 : Import the  Embedding models
> For contextual embedding we can use any embedding model
> BERT, GPT , HuggingFace Embedding, OpenAI EMbedings

In [24]:
import chromadb

In [25]:
from sentence_transformers import SentenceTransformer

In [26]:
from langchain.embeddings  import HuggingFaceEmbeddings

In [27]:
hf_model = HuggingFaceEmbeddings(model_name  = "sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-1727647250.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_model = HuggingFaceEmbeddings(model_name  = "sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 4 Making Database and adding chunks in collection
> we are using the langchain wrapped chromadb
> <br> It provide the functions to add the data in the database and return unique id.

**vdb = Chroma(collection_name="chkdatacoll" ,embedding_function=hf_model)**

In [28]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

# LangChain-managed Chroma vectorstore

In [29]:
# clt.list_collections()
# clt.delete_collection("name of the database")
# clt.delete_collection("chkdatacoll")
# clt.create_collection(name = ,
# embedding_function = )

In [30]:
vdb = Chroma(collection_name="chkdatacoll" ,embedding_function=hf_model)

/tmp/ipython-input-3598755243.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vdb = Chroma(collection_name="chkdatacoll" ,embedding_function=hf_model)


In [31]:
stgids = vdb.add_documents(finalDocs) # This is used to fetch the ids from the chunk of the document.

In [32]:
for doc, sid in zip(finalDocs, stgids):
    doc.metadata["storage_id"] = sid

# Similarity Search: RAG
> RAG works on the basis of Similarity search.
> res = vdb.similarity_search( query = "" , k= no of ansers or data you want)

In [33]:
res = vdb.similarity_search(
    query = "How to build a new habbit?",
    k = 1
)

In [34]:
res[0].page_content

'2.3 · 1019 1.4 · 1020 ConvS2S [8] 25.16 40.46 9.6 · 1018 1.5 · 1020 MoE [26] 26.03 40.56 2.0 · 1019 1.2 · 1020 Deep-Att + PosUnk Ensemble [32] 40.4 8.0 · 1020 GNMT + RL Ensemble [31] 26.30 41.16 1.8 · 1020 1.1 · 1021 ConvS2S Ensemble [8] 26.36 41.29'

# STARTING WITH RAG:

### R --> Retriever retrive the chunks from the vectordb.

## a. Create a retriver

In [35]:
retriver = vdb.as_retriever() #  A fun to to use the vector database as retriver

## B. LLM Instance
    > 1. Using the "Llama-Mimi-8B" model

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline

In [37]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

 #### This dosen't have padding inbuit so we do the manual padding.

In [38]:
google_tokenizer_1 = AutoTokenizer.from_pretrained("google/flan-t5-base")
google_model_1 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [39]:
if google_tokenizer_1.pad_token is None:
     google_tokenizer_1.add_special_tokens({'pad_token':'[PAD]'})#google_tokenizer_1.eos_token})

In [40]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [41]:
google_model_1.resize_token_embeddings(len(google_tokenizer_1))

Embedding(32100, 768)

In [42]:
model.config.pad_token_id = google_tokenizer_1.pad_token_id

#### DeepSeek Model

In [43]:
deepseek_model =  AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-V3.2-Exp")
# model = AutoModelForCausalLM.from_pretrained("llm-jp/Llama-Mimi-8B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# STEP
## Creating generator using pipeline

In [44]:
genrator = pipeline("text2text-generation",
                    model = google_model_1,
                    tokenizer = google_tokenizer_1,
                    max_length = 1000,
                    max_new_tokens = 150)

Device set to use cpu


In [45]:
from langchain.llms import HuggingFacePipeline

>Transforming to the HuggingFace Pipeline to ensure the embeddings and
LLM.

In [46]:
llm = HuggingFacePipeline(pipeline = genrator)

/tmp/ipython-input-1285886572.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = genrator)


## c. Design a Prompt

In [47]:
template = """ Use the context provided to answer the question. if you don't know then answer, say This is out of context.

            Context:
            {context}
            Question: {question}
            Answer:
            """

In [48]:
custom_template = PromptTemplate(
    template = template,
    input_variables = ["context", "question"]
)

In [49]:
custom_template

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" Use the context provided to answer the question. if you don't know then answer, say This is out of context.\n\n            Context:\n            {context}\n            Question: {question}\n            Answer:\n            ")

# Now Connection all things using the RAG chain

In [50]:
rag_chain = (
    {"context": retriver, "question": RunnablePassthrough()} # used to take input from user
    | custom_template
    | llm
    | StrOutputParser() # Format output in a presentable format
)

# Flow:
> Input Question <br>
> Retriver -> Format -> Context<br>
> {Context, question}<br>
> Prompt Template (custom_template)<br>
> LLM (Generate answer)<br>
> StrOutputParser (final otuput string)

# Test

In [51]:
querry = "what is encoder?"

In [52]:
res = rag_chain.invoke(querry)

In [53]:
res

"a stack ofN =6 identical layers. Each layer has two sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position- 2/nFigure 1: The Transformer - model architecture. wise fully connected'), Document(metadata=, page_content='ofN =6 identical layers. In addition to the two sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections'), Document(metadata=, page_content='and"

In [54]:
querry2= "what is decoder?"

In [55]:
res2 = rag_chain.invoke(querry2)

In [57]:
res2

"The decoder is also composed of a stack ofN =6'), Document(metadata=, page_content='function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimensiondmodel = 512. Decoder: The decoder is also composed of a stack ofN =6'), Document(metadata=, page_content='function implemented by the sub-layer itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimensiondmodel = 512"

# Front END

In [ ]:
import gradio as gr
import pypdf
import time
from typing import List, Tuple, Optional

# --- 1. Placeholder for your RAG Pipeline ---
# In a real application, you would initialize your RAG model,
# vector store, and other components here.
class RagPipeline:
    """
    A placeholder class for your RAG pipeline.
    This class should handle PDF processing and question answering.
    """
    def __init__(self):
        self.text_content = ""
        print("Initialized a new RagPipeline instance.")

    def process_pdf(self, pdf_path: str):
        """
        ### TODO: REPLACE THIS ###
        This is the **INGESTION** step.
        Replace this with your actual PDF processing logic.
        This might involve:
        1. Loading the PDF.
        2. Splitting the text into chunks.
        3. Creating vector embeddings for each chunk.
        4. Storing the embeddings in a vector database (FAISS, Chroma, etc.).
        """
        print(f"Processing PDF: {pdf_path}")
        try:
            with open(pdf_path, "rb") as f:
                pdf_reader = pypdf.PdfReader(f)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text()
                self.text_content = text
            print("PDF processed successfully.")
            return True
        except Exception as e:
            print(f"Error processing PDF: {e}")
            return False

    def answer_question(self, question: str) -> str:
        """
        ### TODO: REPLACE THIS ###
        This is the **RETRIEVAL and GENERATION** step.
        Replace this with your actual RAG logic.
        This might involve:
        1. Creating an embedding for the user's question.
        2. Searching your vector database for the most relevant text chunks.
        3. Combining the question and the retrieved chunks into a prompt.
        4. Sending the prompt to your LLM.
        5. Returning the LLM's generated answer.
        """
        print(f"Answering question: {question}")
        # Simulate a RAG process by searching for the question in the text
        if self.text_content and question.lower() in self.text_content.lower():
            return f"Based on the document, I found information related to '{question}'. (This is a simulated RAG response)."
        elif self.text_content:
            return "I couldn't find a specific answer to your question in the document. (This is a simulated RAG response)."
        else:
            return "The document has not been processed yet. Please upload a PDF first."

# --- 2. Gradio Functions ---
# These functions link your RAG pipeline to the Gradio UI.

def process_document(file, progress=gr.Progress(track_tqdm=True)):
    """
    Gradio function to handle the PDF upload and processing.
    It creates and returns a new RAG pipeline instance for the session.
    """
    if file is None:
        return None, gr.update(visible=True, value="Please upload a PDF to begin.")

    progress(0, desc="Starting PDF processing...")
    time.sleep(0.5) # Simulate startup time

    # Initialize a new RAG pipeline for this document/session
    rag_pipeline = RagPipeline()

    progress(0.2, desc="Extracting text from PDF...")
    # The file object has a .name attribute which is a temporary path
    success = rag_pipeline.process_pdf(file.name)

    if not success:
        return None, gr.update(visible=True, value="Failed to process the PDF. Please try another file.")

    progress(1, desc="Processing complete!")
    time.sleep(0.5)

    # Return the initialized pipeline to be stored in state
    return rag_pipeline, gr.update(visible=True, value="PDF processed successfully! You can now ask questions.")

def add_message_to_history(user_message: str, history: List[Tuple[str, str]]):
    """Adds the user's message to the chat history for immediate display."""
    return history + [[user_message, None]]

def chat_responder(history: List[Tuple[str, str]], rag_pipeline: Optional[RagPipeline]):
    """
    The main chat function. It takes the user's question and the RAG pipeline,
    gets an answer, and streams it back to the chatbot UI.
    """
    if rag_pipeline is None:
        yield history[:-1], "Please upload a PDF before asking questions."
        return

    user_question = history[-1][0]
    bot_response = ""
    history[-1][1] = ""

    # Get the answer from your RAG pipeline
    answer = rag_pipeline.answer_question(user_question)

    # Stream the response for a better user experience
    for char in answer:
        bot_response += char
        history[-1][1] = bot_response
        time.sleep(0.02)
        yield history

# --- 3. Gradio Interface Definition ---

with gr.Blocks(theme=gr.themes.Soft(primary_hue="sky"), title="RAG Chatbot") as demo:
    # State component to store the RAG pipeline instance for the session
    rag_state = gr.State()

    gr.Markdown(
        """
        # 📄 RAG-Based Chatbot 🤖
        Upload a PDF document and ask any questions you have about its content.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            upload_button = gr.UploadButton(
                "📁 Upload PDF",
                file_types=[".pdf"],
                variant="primary"
            )
            process_status = gr.Markdown(visible=False)

        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbot",
                bubble_full_width=False,
                avatar_images=(None, "https://gradio.app/images/logo.svg"),
                height=500,
                label="Chat about your Document"
            )
            chat_input = gr.Textbox(
                show_label=False,
                placeholder="Ask a question about the document...",
                lines=1
            )
            clear_button = gr.ClearButton([chat_input, chatbot], value="✨ Clear Chat")

    # --- Event Listeners ---

    # 1. When a PDF is uploaded
    upload_button.upload(
        fn=process_document,
        inputs=[upload_button],
        outputs=[rag_state, process_status],
        show_progress="full"
    )

    # 2. When the user submits a question (via Enter key)
    chat_input.submit(
        fn=add_message_to_history,
        inputs=[chat_input, chatbot],
        outputs=[chatbot],
        queue=False
    ).then(
        fn=chat_responder,
        inputs=[chatbot, rag_state],
        outputs=[chatbot],
    ).then(
        lambda: gr.update(value=""), None, [chat_input], queue=False
    )

if __name__ == "__main__":
    demo.launch(debug=True)

/tmp/ipython-input-1258417098.py:144: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1258417098.py:144: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d13742869a3b71f2a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2127, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1904, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/components/chatbot.py", line 632, in postproce

Initialized a new RagPipeline instance.
Processing PDF: /tmp/gradio/79ad9b08f252b07e9d72fe0914fc214006f770ed3b65e318b9115a95af914497/4A.pdf
PDF processed successfully.
Initialized a new RagPipeline instance.
Processing PDF: /tmp/gradio/79ad9b08f252b07e9d72fe0914fc214006f770ed3b65e318b9115a95af914497/4A.pdf
PDF processed successfully.
Answering question: HI
Answering question: HLO HOW ARE  YOU?
